# Hockey Reference Data Processing
We prepare the data according to two models:
* A minimal model with regular stats (based on https://pub.towardsai.net/how-to-win-your-nhl-pool-without-even-trying-42bc03b9659)
* An advanced model with advanced stats

In [1]:
#Import libraries and modules
import dill
import time
import pandas as pd

import numpy as np
import math

# Relevant Functions

In [2]:
# Convert nan's to zero
def nan_to_zero(data):
    for i,r in enumerate(data):
        for j,c in enumerate(r):
            if not isinstance(c, str):
                if math.isnan(c):
                    data[i][j]=0
        #print(f'Total nan after loop for year {year}: {pd.DataFrame(data[year]).isna().sum().sum()}')
    
    return data

In [3]:
# Use the following only on the training data, not the validation one
# We will mostly use Standardize
# Should we normalize/standardize the Fantasy Points column? I think so...
def data_Normalize(data, target_col):
    newData=data.copy()
    
    for c in list(target_col):
        # First exception shouldn't happen since we deal with strings before
        if (not isinstance(data.loc[ data.index[0],c], str)) :
            minval=np.min(data[c])
            maxval=np.max(data[c])
            newData[c]=(data[c]-minval)/(maxval-minval)
            
    return newData

def data_Standardize(data, target_col):
    newData=data.copy()
    
    for c in list(data.columns):
        # First exception shouldn't happen since we deal with strings before
        if (not isinstance(data.loc[ data.index[0],c], str)) :
            avg=np.mean(data[c])
            std=np.std(data[c])
            newData[c]=(data[c]-avg)/std
            
    return newData

# Column Full Data Dictionary

In [4]:
dill_file = open("all_col_names", "rb")
col_dict = dill.load(dill_file)
dill_file.close()

# Import data

In [5]:
dill_file = open("dataYear_08_22", "rb")
dataYear = dill.load(dill_file)
dill_file.close()

# Organize data for minimal model
We process the database by concatenating data for years 20XX and 20XX-1, and we produce a target dataFrame with data from year 20XX+1. In this section, we prepare data for a minimal model similar to 
https://pub.towardsai.net/how-to-win-your-nhl-pool-without-even-trying-42bc03b9659

In [6]:
col_dict

{0: ('Name', 'Name'),
 1: ('Unnamed: -2_level_0', 'Fantasy Points'),
 2: ('Unnamed: -1_level_0', 'Position'),
 3: ('Unnamed: 0_level_0', 'Season'),
 4: ('Unnamed: 1_level_0', 'Age'),
 5: ('Unnamed: 2_level_0', 'Tm'),
 6: ('Unnamed: 3_level_0', 'Lg'),
 7: ('Unnamed: 4_level_0', 'GP'),
 8: ('Scoring', 'G'),
 9: ('Scoring', 'A'),
 10: ('Scoring', 'PTS'),
 11: ('Unnamed: 8_level_0', '+/-'),
 12: ('Unnamed: 9_level_0', 'PIM'),
 13: ('Goals', 'EV'),
 14: ('Goals', 'PP'),
 15: ('Goals', 'SH'),
 16: ('Goals', 'GW'),
 17: ('Assists', 'EV'),
 18: ('Assists', 'PP'),
 19: ('Assists', 'SH'),
 20: ('Shots', 'S'),
 21: ('Shots', 'S%'),
 22: ('Shots', 'TSA'),
 23: ('Ice Time', 'TOI'),
 24: ('Ice Time', 'ATOI'),
 25: ('Unnamed: 22_level_0', 'FOW'),
 26: ('Unnamed: 23_level_0', 'FOL'),
 27: ('Unnamed: 24_level_0', 'FO%'),
 28: ('Unnamed: 25_level_0', 'BLK'),
 29: ('Unnamed: 26_level_0', 'HIT'),
 30: ('Unnamed: 27_level_0', 'TK'),
 31: ('Unnamed: 28_level_0', 'GV'),
 32: ('Unnamed: 29_level_0', 'Awards')

In [7]:
# For players having played on multiple teams, keep stats of the total season only
def keep_tot_stat(data):
    # Data input is the entire dataFrame, no filtering/preprocessing
    seen = set()
    dupes = []
    for i,x in enumerate(list(data[0].values)):
        if x in seen:
            dupes.append(x)
        else:
            seen.add(x)
    to_remove=[]
    for i,x in enumerate(list(data[0].values)):
        if (x in dupes) and (data[5][i]!='TOT'):
            to_remove.append(i)
    obj_to_remove=[data.index[j] for j in to_remove]
    newData=data.drop(obj_to_remove)
    
    # Remove duplicates according to column=0=Name
    newData2=newData.drop_duplicates(subset=0)
    
    newData2.index=range(len(newData2.index))
    
    return newData2

In [41]:
# This function (named after the author of https://pub.towardsai.net/how-to-win-your-nhl-pool-without-even-trying-42bc03b9659) 
# seeks to select a minimal amount of data to make predictions
# i.e. Height (nan), Weight (nan), Age (4), Position(2), GPG (304), PPG (306), Hits/game (29)/(7), Shots/game (309)
# PIM/game (308), TOI/game (24), fractions GP/82 (7)/82, Total +/- per game (11)/(7)
# We will ignore Height/Weight, and Position will be a binary encoding (0:D, 1:F).
# We further add Fantasy Points since this is our primary objective, and shots blocked/game (28) since we know it to be a dependent variable
yan_col_names=[0,1,2,4,7,306,304,309,308,29,28,24,11]
def Yan_NN_data(data):
    # Takes dataYear as input
    nn_XdataYear={}
    nn_YdataYear={}
    
    for year in list(data.keys())[1:-1]:
        nn_XdataYear[year]=[]
        nn_YdataYear[year]=[]
        
        if year != 2021:
            # Index 1 = Current Year, 0 = Previous Year, 2 = Next Year
            # Remove duplicate players and keep total season stat
            newData={}
            newData[1]=keep_tot_stat(data[year])
            newData[0]=keep_tot_stat(data[year-1])
            newData[2]=keep_tot_stat(data[year+1])

            subselection={}
            # Name(0), FP/GP(1)/(7), Position(2), Age(4), GP/82(7)/82, PPG(306), GPG(304), SPG(309), 
            # PIM/game(308), HPG(29)/(7), BLKPG(28)/(7), TOI/game(24), +/-(11)
            column_names=[0,1,2,4,7,306,304,309,308,29,28,24,11]
            subselection[1]=pd.concat([newData[1][0], newData[1][1]/newData[1][7], newData[1][2], newData[1][4], newData[1][7]/82, newData[1][306], newData[1][304], newData[1][309], 
                                         newData[1][308], newData[1][29]/newData[1][7], newData[1][28]/newData[1][7], newData[1][24], newData[1][11]/82 ], axis=1)
            subselection[1].columns=column_names
            subselection[1].index=range(len(subselection[1].index))

            subselection[0]=pd.concat([newData[0][0], newData[0][1]/newData[0][7], newData[0][2], newData[0][4], newData[0][7]/82, newData[0][306], newData[0][304], newData[0][309], 
                                         newData[0][308], newData[0][29]/newData[0][7], newData[0][28]/newData[0][7], newData[0][24], newData[0][11]/82 ], axis=1)
            subselection[0].columns=column_names
            subselection[0].index=range(len(subselection[0].index))

            subselection[2]=pd.concat([newData[2][0], newData[2][1]/newData[2][7], newData[2][2], newData[2][4], newData[2][7]/82, newData[2][306], newData[2][304], newData[2][309], 
                                         newData[2][308], newData[2][29]/newData[2][7], newData[2][28]/newData[2][7], newData[2][24], newData[2][11]/82 ], axis=1)
            subselection[2].columns=column_names
            subselection[2].index=range(len(subselection[2].index))

            # Select desirable players and replace nan by 0
            minPointProduction=0.2

            for r in subselection[1].index:
                player_name=subselection[1][0][r]
                # Player has to have played in 3rd season
                if player_name in list(subselection[2][0]):
                    ind_next = list(subselection[2][0]).index(player_name)
                    # Catalogue players
                    if ( #Played in the previous season or scored more than min point production
                        (player_name in list(subselection[0][0]) or  subselection[1].loc[r,306]>minPointProduction) 
                        # Minimum FP/game production in current season
                        and subselection[1].loc[r,1]>0.5
                        # Played minmum games in current season
                        and subselection[1].loc[r,7]>0.15
                        # Played min games in next season
                        and subselection[2].loc[ind_next,7]>0.15
                        ):
                        curr_year=subselection[1].loc[r,:].values

                        # Check if player played in the previous season
                        if player_name in list(subselection[0][0]):
                            # Find instances where player appears in the previous year's data
                            ind=list(subselection[0][0]).index(player_name)
                            target=subselection[0].loc[ind,:].values

                            prev_year=target

                        elif subselection[1].loc[r,306]>minPointProduction:
                            prev_year=np.zeros(np.shape(curr_year))
                            prev_year[7]=0

                        targetYFP=subselection[2].loc[list(subselection[2][0]).index(player_name),1]/subselection[2].loc[list(subselection[2][0]).index(player_name),7]
                        targetYPPG=subselection[2].loc[list(subselection[2][0]).index(player_name),306]

                        if (not math.isnan(targetYFP)) or (not math.isnan(targetYPPG)):
                            nn_XdataYear[year].append(np.concatenate((curr_year, prev_year)))
                            nn_YdataYear[year].append([targetYFP, targetYPPG])
        else:
            newData={}
            newData[1]=keep_tot_stat(data[year])
            newData[0]=keep_tot_stat(data[year-1])

            subselection={}
            # Name(0), FP/GP(1)/(7), Position(2), Age(4), GP/82(7)/82, PPG(306), GPG(304), SPG(309), 
            # PIM/game(308), HPG(29)/(7), BLKPG(28)/(7), TOI/game(24), +/-(11)
            column_names=[0,1,2,4,7,306,304,309,308,29,28,24,11]
            subselection[1]=pd.concat([newData[1][0], newData[1][1]/newData[1][7], newData[1][2], newData[1][4], newData[1][7]/82, newData[1][306], newData[1][304], newData[1][309], 
                                         newData[1][308], newData[1][29]/newData[1][7], newData[1][28]/newData[1][7], newData[1][24], newData[1][11]/82 ], axis=1)
            subselection[1].columns=column_names
            subselection[1].index=range(len(subselection[1].index))

            subselection[0]=pd.concat([newData[0][0], newData[0][1]/newData[0][7], newData[0][2], newData[0][4], newData[0][7]/82, newData[0][306], newData[0][304], newData[0][309], 
                                         newData[0][308], newData[0][29]/newData[0][7], newData[0][28]/newData[0][7], newData[0][24], newData[0][11]/82 ], axis=1)
            subselection[0].columns=column_names
            subselection[0].index=range(len(subselection[0].index))

            # Select desirable players and replace nan by 0
            minPointProduction=0.2

            for r in subselection[1].index:
                player_name=subselection[1][0][r]
                # Catalogue players
                if ( #Played in the previous season or scored more than min point production
                    (player_name in list(subselection[0][0]) or  subselection[1].loc[r,306]>minPointProduction) 
                    # Minimum FP/game production in current season
                    and subselection[1].loc[r,1]>0.5
                    # Played minmum games in current season
                    and subselection[1].loc[r,7]>0.15
                    ):
                    curr_year=subselection[1].loc[r,:].values

                    # Check if player played in the previous season
                    if player_name in list(subselection[0][0]):
                        # Find instances where player appears in the previous year's data
                        ind=list(subselection[0][0]).index(player_name)
                        target=subselection[0].loc[ind,:].values

                        prev_year=target

                    elif subselection[1].loc[r,306]>minPointProduction:
                        prev_year=np.zeros(np.shape(curr_year))
                        prev_year[7]=0

                    nn_XdataYear[year].append(np.concatenate((curr_year, prev_year)))

    return nn_XdataYear, nn_YdataYear

In [42]:
print(dataYear.keys())
start_time=time.time()
nn_XdataYear,nn_YdataYear=Yan_NN_data(dataYear)
print(f'Finished preparing data in {time.time()-start_time}s')
print(nn_XdataYear.keys())

dict_keys([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])
Finished preparing data in 3.736001968383789s
dict_keys([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])


In [43]:
display(pd.DataFrame(nn_XdataYear[2011]))
display(pd.DataFrame(nn_YdataYear[2011]))

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,Claude Giroux,7.659756,1.0,23.0,1.000000,0.93,0.30,2.06,0.57,1.426829,...,22.0,1.000000,0.57,0.20,1.77,0.28,0.926829,0.585366,16:37,-0.109756
1,Daniel Briere,8.420779,1.0,33.0,0.939024,0.88,0.44,3.19,1.13,0.467532,...,32.0,0.914634,0.71,0.35,2.57,0.95,0.706667,0.226667,16:35,-0.024390
2,Jeff Carter,9.581250,1.0,26.0,0.975610,0.83,0.45,4.19,0.49,0.812500,...,25.0,0.902439,0.82,0.45,4.31,0.51,0.810811,0.459459,19:18,0.024390
3,Mike Richards2,7.562963,1.0,25.0,0.987805,0.81,0.28,2.27,0.77,1.283951,...,24.0,1.000000,0.76,0.38,2.89,0.96,1.768293,0.780488,20:24,-0.024390
4,Ville Leino,5.497531,1.0,27.0,0.987805,0.65,0.23,1.44,0.27,0.432099,...,26.0,0.670732,0.20,0.11,1.40,0.18,0.345455,0.309091,13:05,-0.097561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,Jeff Woywitka,3.552381,0.0,27.0,0.768293,0.17,0.03,1.14,0.38,0.682540,...,26.0,0.439024,0.08,0.00,1.22,0.31,0.777778,1.166667,14:06,-0.073171
574,Nicklas Grossmann,2.850847,0.0,26.0,0.719512,0.17,0.02,0.64,0.59,2.728814,...,25.0,0.865854,0.10,0.00,0.82,0.45,2.112676,1.408451,19:11,-0.036585
575,Tom Wandell,2.314667,1.0,24.0,0.914634,0.12,0.09,1.25,0.19,0.866667,...,23.0,0.609756,0.30,0.10,1.70,0.28,0.880000,0.800000,13:52,0.024390
576,Toby Petersen,1.703333,1.0,32.0,0.731707,0.10,0.03,0.97,0.13,0.933333,...,31.0,0.951220,0.19,0.12,1.41,0.08,1.192308,0.551282,10:55,0.036585


,0,1
0,10.757227,1.21
1,7.440245,0.70
2,9.829157,0.62
3,6.557305,0.59
4,3.451775,0.35
...,...,...
573,7.165158,0.22
574,3.839445,0.15
575,2.921566,0.21
576,3.536621,0.13


In [44]:
# Column dictionary
yan_col_names=[0,1,2,4,7,306,304,309,308,29,28,24,11]
yan_col_dict={}
for i,j in enumerate(yan_col_names):
    yan_col_dict[i]=('S1',list(col_dict[j])[1])
for i,j in enumerate(yan_col_names):
    yan_col_dict[i+len(yan_col_names)]=('S2',list(col_dict[j])[1])
yan_col_dict

{0: ('S1', 'Name'),
 1: ('S1', 'Fantasy Points'),
 2: ('S1', 'Position'),
 3: ('S1', 'Age'),
 4: ('S1', 'GP'),
 5: ('S1', 'PTS'),
 6: ('S1', 'G'),
 7: ('S1', 'S'),
 8: ('S1', 'PIM'),
 9: ('S1', 'HIT'),
 10: ('S1', 'BLK'),
 11: ('S1', 'ATOI'),
 12: ('S1', '+/-'),
 13: ('S2', 'Name'),
 14: ('S2', 'Fantasy Points'),
 15: ('S2', 'Position'),
 16: ('S2', 'Age'),
 17: ('S2', 'GP'),
 18: ('S2', 'PTS'),
 19: ('S2', 'G'),
 20: ('S2', 'S'),
 21: ('S2', 'PIM'),
 22: ('S2', 'HIT'),
 23: ('S2', 'BLK'),
 24: ('S2', 'ATOI'),
 25: ('S2', '+/-')}

In [45]:
len(yan_col_names)

13

In [46]:
def Yan_data_Clean(data, yTarget):
    rows_to_drop=[]
    for r in data.index:
        # Convert ATOI string to integer of seconds, measured in hours (60mins)
        if isinstance(data.loc[r,11], str):
            data.loc[r,11]=sum([a*b for a,b in zip([60,1], map(int, data.loc[r,11].split(':')))])/3600
        else:
            rows_to_drop.append(r)
        if isinstance(data.loc[r,24], str):
            data.loc[r,24]=sum([a*b for a,b in zip([60,1], map(int, data.loc[r,24].split(':')))])/3600
        else:
            rows_to_drop.append(r)
    
    data=data.drop(13, axis=1) # Name duplicate
    data=pd.DataFrame(nan_to_zero(np.array(data)))
    data.insert(0, 'Target PPG', pd.DataFrame(yTarget)[1])
    data.insert(0, 'Target FPPG', pd.DataFrame(yTarget)[0])
    data=data.drop(rows_to_drop, axis=0)
    
    return data

In [47]:
pd.DataFrame(nn_XdataYear[2009]).columns

RangeIndex(start=0, stop=26, step=1)

In [48]:
# Obtain cleaned data
start_time=time.time()
all_cleaned_data=Yan_data_Clean(pd.DataFrame(nn_XdataYear[2009]),nn_YdataYear[2009])
for i in range(2010,2021):
    print(i)
    temp=Yan_data_Clean(pd.DataFrame(nn_XdataYear[i]),nn_YdataYear[i])
    all_cleaned_data=pd.concat([all_cleaned_data, temp])
all_cleaned_data.index=range(len(all_cleaned_data.index))
print(f'Finished cleaning data in {time.time()-start_time}s')

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Finished cleaning data in 1.1729187965393066s


In [49]:
# Check that there are no nan's
print(all_cleaned_data.isna().sum().sum())
for i,r in enumerate(np.array(all_cleaned_data)):
    for j,c in enumerate(r):
        if not isinstance(c, str):
            if math.isnan(c):
                print(i,j)
                print(r)

0


In [15]:
# Export data
import dill
dill_file=open("all_data_min_v1","wb")
dill_file.write(dill.dumps(all_cleaned_data))
dill_file.close()

In [54]:
all_cleaned_test=Yan_data_Clean(pd.DataFrame(nn_XdataYear[2021]),nn_YdataYear[2020])
dill_file=open("all_data_min_test","wb")
dill_file.write(dill.dumps(all_cleaned_test))
dill_file.close()

# Advanced Model

In [17]:
# For players having played on multiple teams, keep stats of the total season only
def keep_tot_stat(data):
    # Data input is the entire dataFrame, no filtering/preprocessing
    seen = set()
    dupes = []
    for i,x in enumerate(list(data[0].values)):
        if x in seen:
            dupes.append(x)
        else:
            seen.add(x)
    to_remove=[]
    for i,x in enumerate(list(data[0].values)):
        if (x in dupes) and (data[5][i]!='TOT'):
            to_remove.append(i)
    obj_to_remove=[data.index[j] for j in to_remove]
    newData=data.drop(obj_to_remove)
    
    # Remove duplicates according to column=0=Name
    newData2=newData.drop_duplicates(subset=0)
    
    newData2.index=range(len(newData2.index))
    
    return newData2

In [18]:
# Advanced stats model. We handpick stats that we deem relevant.
def nn_data_selection_01(data):
        # Takes dataYear as input
    nn_XdataYear={}
    nn_YdataYear={}
    
    for year in list(data.keys())[1:-1]:
        nn_XdataYear[year]=[]
        nn_YdataYear[year]=[]
        
        if year != 2021:
            # Index 1 = Current Year, 0 = Previous Year, 2 = Next Year
            # Remove duplicate players and keep total season stat
            newData={}
            newData[1]=keep_tot_stat(data[year])
            newData[0]=keep_tot_stat(data[year-1])
            newData[2]=keep_tot_stat(data[year+1])

            subselection={}
            # Name(0), FP/GP(1)/(7), Position(2), Age(4), GP/82(7)/82, TOI/game(24), +/-(11), PPG(306), GPG(304), SPG(309), TSAGP(39)/(7), SThr%(40)/100
            # PIM/game(308), HPG(29)/(7), BLKPG(28)/(7), TKPG (46)/(7), GVPG (47)/(7), 
            # CF% (126)/100, CF% rel (127)/100, FF% (130)/100, FF% rel (131)/100, oiSH% (133)/100, oZS% (137)/100
            # PTS/60 (223), S/60 (224), oiGF/60 (225), C/60 (229), F/60 (232), PS (321)
            column_names=[0,3,1,2,4,7,24,11,306,304,309,39,40,308,29,28,46,47,
                             126,127,130,131,133,137,223,224,225,229,232,321]
            subselection[1]=pd.concat([newData[1][0], newData[1][3].apply(lambda x:int(x[0:4])-2000+1) , newData[1][1]/newData[1][7], newData[1][2], newData[1][4], newData[1][7]/82, newData[1][24], newData[1][11]/82 , newData[1][306], newData[1][304], newData[1][309], newData[1][39]/newData[1][7], newData[1][40]/100, 
                                         newData[1][308], newData[1][29]/newData[1][7], newData[1][28]/newData[1][7], newData[1][46]/newData[1][7], newData[1][47]/newData[1][7],
                                         newData[1][126]/100, newData[1][127]/100, newData[1][130]/100, newData[1][131]/100, newData[1][133]/100, newData[1][137]/100,
                                         newData[1][223], newData[1][224], newData[1][225], newData[1][229], newData[1][232], newData[1][321] ], axis=1)
            subselection[1].columns=column_names
            subselection[1].index=range(len(subselection[1].index))

            subselection[0]=pd.concat([newData[0][0], newData[0][3].apply(lambda x:int(x[0:4])-2000+1) , newData[0][1]/newData[0][7], newData[0][2], newData[0][4], newData[0][7]/82, newData[0][24], newData[0][11]/82, newData[0][306], newData[0][304], newData[0][309], newData[0][39]/newData[0][7], newData[0][40]/100,
                                         newData[0][308], newData[0][29]/newData[0][7], newData[0][28]/newData[0][7], newData[0][46]/newData[0][7], newData[0][47]/newData[0][7],
                                         newData[0][126]/100, newData[0][127]/100, newData[0][130]/100, newData[0][131]/100, newData[0][133]/100, newData[0][137]/100,
                                         newData[0][223], newData[0][224], newData[0][225], newData[0][229], newData[0][232], newData[0][321] ], axis=1)
            subselection[0].columns=column_names
            subselection[0].index=range(len(subselection[0].index))

            subselection[2]=pd.concat([newData[2][0], newData[2][3].apply(lambda x:int(x[0:4])-2000+1) , newData[2][1]/newData[2][7], newData[2][2], newData[2][4], newData[2][7]/82, newData[2][24], newData[2][11]/82, newData[2][306], newData[2][304], newData[2][309], newData[2][39]/newData[2][7], newData[2][40]/100,
                                         newData[2][308], newData[2][29]/newData[2][7], newData[2][28]/newData[2][7], newData[2][46]/newData[2][7], newData[2][47]/newData[2][7],
                                         newData[2][126]/100, newData[2][127]/100, newData[2][130]/100, newData[2][131]/100, newData[2][133]/100, newData[2][137]/100,
                                         newData[2][223], newData[2][224], newData[2][225], newData[2][229], newData[2][232], newData[2][321] ], axis=1)
            subselection[2].columns=column_names
            subselection[2].index=range(len(subselection[2].index))

            # Select desirable players and replace nan by 0
            minPointProduction=0.2

            for r in subselection[1].index:
                player_name=subselection[1][0][r]
                # Player has to have played in 3rd season
                if player_name in list(subselection[2][0]):
                    ind_next = list(subselection[2][0]).index(player_name)
                    # Catalogue players
                    if ( #Played in the previous season or scored more than min point production
                        (player_name in list(subselection[0][0]) or  subselection[1].loc[r,306]>minPointProduction) 
                        # Minimum FP/game production in current season
                        and subselection[1].loc[r,1]>0.5
                        # Played minmum games in current season
                        and subselection[1].loc[r,7]>0.15
                        # Played min games in next season
                        and subselection[2].loc[ind_next,7]>0.15
                        ):
                        curr_year=subselection[1].loc[r,:].values

                        # Check if player played in the previous season
                        if player_name in list(subselection[0][0]):
                            # Find instances where player appears in the previous year's data
                            ind=list(subselection[0][0]).index(player_name)
                            target=subselection[0].loc[ind,:].values

                            prev_year=target

                        elif subselection[1].loc[r,306]>minPointProduction:
                            prev_year=np.zeros(np.shape(curr_year))
                            prev_year[7]=0

                        targetYFP=subselection[2].loc[list(subselection[2][0]).index(player_name),1]/subselection[2].loc[list(subselection[2][0]).index(player_name),7]
                        targetYPPG=subselection[2].loc[list(subselection[2][0]).index(player_name),306]

                        if (not math.isnan(targetYFP)) or (not math.isnan(targetYPPG)):
                            nn_XdataYear[year].append(np.concatenate((curr_year, prev_year)))
                            nn_YdataYear[year].append([targetYFP, targetYPPG])
        else:
            newData={}
            newData[1]=keep_tot_stat(data[year])
            newData[0]=keep_tot_stat(data[year-1])

            subselection={}
            # Name(0), FP/GP(1)/(7), Position(2), Age(4), GP/82(7)/82, TOI/game(24), +/-(11), PPG(306), GPG(304), SPG(309), TSAGP(39)/(7), SThr%(40)/100
            # PIM/game(308), HPG(29)/(7), BLKPG(28)/(7), TKPG (46)/(7), GVPG (47)/(7), 
            # CF% (126)/100, CF% rel (127)/100, FF% (130)/100, FF% rel (131)/100, oiSH% (133)/100, oZS% (137)/100
            # PTS/60 (223), S/60 (224), oiGF/60 (225), C/60 (229), F/60 (232), PS (321)
            column_names=[0,3,1,2,4,7,24,11,306,304,309,39,40,308,29,28,46,47,
                             126,127,130,131,133,137,223,224,225,229,232,321]
            subselection[1]=pd.concat([newData[1][0], newData[1][3].apply(lambda x:int(x[0:4])-2000+1) , newData[1][1]/newData[1][7], newData[1][2], newData[1][4], newData[1][7]/82, newData[1][24], newData[1][11]/82 , newData[1][306], newData[1][304], newData[1][309], newData[1][39]/newData[1][7], newData[1][40]/100, 
                                         newData[1][308], newData[1][29]/newData[1][7], newData[1][28]/newData[1][7], newData[1][46]/newData[1][7], newData[1][47]/newData[1][7],
                                         newData[1][126]/100, newData[1][127]/100, newData[1][130]/100, newData[1][131]/100, newData[1][133]/100, newData[1][137]/100,
                                         newData[1][223], newData[1][224], newData[1][225], newData[1][229], newData[1][232], newData[1][321] ], axis=1)
            subselection[1].columns=column_names
            subselection[1].index=range(len(subselection[1].index))

            subselection[0]=pd.concat([newData[0][0], newData[0][3].apply(lambda x:int(x[0:4])-2000+1) , newData[0][1]/newData[0][7], newData[0][2], newData[0][4], newData[0][7]/82, newData[0][24], newData[0][11]/82, newData[0][306], newData[0][304], newData[0][309], newData[0][39]/newData[0][7], newData[0][40]/100,
                                         newData[0][308], newData[0][29]/newData[0][7], newData[0][28]/newData[0][7], newData[0][46]/newData[0][7], newData[0][47]/newData[0][7],
                                         newData[0][126]/100, newData[0][127]/100, newData[0][130]/100, newData[0][131]/100, newData[0][133]/100, newData[0][137]/100,
                                         newData[0][223], newData[0][224], newData[0][225], newData[0][229], newData[0][232], newData[0][321] ], axis=1)
            subselection[0].columns=column_names
            subselection[0].index=range(len(subselection[0].index))

            # Select desirable players and replace nan by 0
            minPointProduction=0.2

            for r in subselection[1].index:
                player_name=subselection[1][0][r]
                # Catalogue players
                if ( #Played in the previous season or scored more than min point production
                    (player_name in list(subselection[0][0]) or  subselection[1].loc[r,306]>minPointProduction) 
                    # Minimum FP/game production in current season
                    and subselection[1].loc[r,1]>0.5
                    # Played minmum games in current season
                    and subselection[1].loc[r,7]>0.15
                    ):
                    curr_year=subselection[1].loc[r,:].values

                    # Check if player played in the previous season
                    if player_name in list(subselection[0][0]):
                        # Find instances where player appears in the previous year's data
                        ind=list(subselection[0][0]).index(player_name)
                        target=subselection[0].loc[ind,:].values

                        prev_year=target

                    elif subselection[1].loc[r,306]>minPointProduction:
                        prev_year=np.zeros(np.shape(curr_year))
                        prev_year[7]=0

                    nn_XdataYear[year].append(np.concatenate((curr_year, prev_year)))

    return nn_XdataYear, nn_YdataYear

In [19]:
start_time=time.time()
nn_XdataYear,nn_YdataYear=nn_data_selection_01(dataYear)
print(f'Finished preparing data in {time.time()-start_time}s')

Finished preparing data in 3.8988900184631348s


In [20]:
display(pd.DataFrame(nn_XdataYear[2011]))
display(pd.DataFrame(nn_YdataYear[2011]))

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,Claude Giroux,11,7.659756,1.0,23.0,1.000000,19:24,0.243902,0.93,0.30,...,0.576,0.055,0.087,0.599,2.1,6.4,2.9,14.0,12.2,4.3
1,Daniel Briere,11,8.420779,1.0,33.0,0.939024,18:19,0.243902,0.88,0.44,...,0.601,0.082,0.094,0.670,2.5,9.3,3.9,26.5,19.3,6.5
2,Jeff Carter,11,9.581250,1.0,26.0,0.975610,18:15,0.329268,0.83,0.45,...,0.571,0.050,0.096,0.588,2.6,13.4,3.7,16.1,13.3,8.0
3,Mike Richards2,11,7.562963,1.0,25.0,0.987805,18:53,0.134146,0.81,0.28,...,0.559,0.027,0.096,0.526,2.2,8.5,3.2,11.8,10.0,7.3
4,Ville Leino,11,5.497531,1.0,27.0,0.987805,16:01,0.170732,0.65,0.23,...,0.591,0.075,0.062,0.636,0.9,6.4,2.1,22.4,15.4,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,Jeff Woywitka,11,3.552381,0.0,27.0,0.768293,17:57,0.060976,0.17,0.03,...,0.475,-0.051,0.062,0.547,0.4,5.3,1.8,-4.1,-4.5,0.2
574,Nicklas Grossmann,11,2.850847,0.0,26.0,0.719512,18:12,0.085366,0.17,0.02,...,0.424,-0.111,0.098,0.400,0.3,2.6,2.6,-17.1,-12.6,1.5
575,Tom Wandell,11,2.314667,1.0,24.0,0.914634,11:45,-0.060976,0.12,0.09,...,0.468,-0.066,0.067,0.458,1.3,7.5,1.9,-7.1,-5.1,1.0
576,Toby Petersen,11,1.703333,1.0,32.0,0.731707,10:02,-0.085366,0.10,0.03,...,0.452,-0.068,0.065,0.497,1.1,7.9,1.8,-13.6,-8.1,1.1


,0,1
0,10.757227,1.21
1,7.440245,0.70
2,9.829157,0.62
3,6.557305,0.59
4,3.451775,0.35
...,...,...
573,7.165158,0.22
574,3.839445,0.15
575,2.921566,0.21
576,3.536621,0.13


In [21]:
#Clean the data and remove nan's
temp={}
for key in nn_XdataYear.keys():
    nn_XdataYear[key]=nan_to_zero(nn_XdataYear[key])
    print(f'Total nan after loop for year {key}: {pd.DataFrame(nn_XdataYear[key]).isna().sum().sum()}')

Total nan after loop for year 2009: 0
Total nan after loop for year 2010: 0
Total nan after loop for year 2011: 0
Total nan after loop for year 2012: 0
Total nan after loop for year 2013: 0
Total nan after loop for year 2014: 0
Total nan after loop for year 2015: 0
Total nan after loop for year 2016: 0
Total nan after loop for year 2017: 0
Total nan after loop for year 2018: 0
Total nan after loop for year 2019: 0
Total nan after loop for year 2020: 0
Total nan after loop for year 2021: 0


In [22]:
# Create a sub-dictionary for the selected columns
sel01_col=[0,3,1,2,4,7,24,11,306,304,309,39,40,308,29,28,46,47,
                             126,127,130,131,133,137,223,224,225,229,232,321]
sel_col_dict={}
for i,j in enumerate(sel01_col):
    sel_col_dict[i]=('S1',list(col_dict[j])[1])
for i,j in enumerate(sel01_col):
    sel_col_dict[i+len(sel01_col)]=('S2',list(col_dict[j])[1])
sel_col_dict

{0: ('S1', 'Name'),
 1: ('S1', 'Season'),
 2: ('S1', 'Fantasy Points'),
 3: ('S1', 'Position'),
 4: ('S1', 'Age'),
 5: ('S1', 'GP'),
 6: ('S1', 'ATOI'),
 7: ('S1', '+/-'),
 8: ('S1', 'PTS'),
 9: ('S1', 'G'),
 10: ('S1', 'S'),
 11: ('S1', 'TSA'),
 12: ('S1', 'SThr%'),
 13: ('S1', 'PIM'),
 14: ('S1', 'HIT'),
 15: ('S1', 'BLK'),
 16: ('S1', 'TK'),
 17: ('S1', 'GV'),
 18: ('S1', 'CF%'),
 19: ('S1', 'CF% rel'),
 20: ('S1', 'FF%'),
 21: ('S1', 'FF% rel'),
 22: ('S1', 'oiSH%'),
 23: ('S1', 'oZS%'),
 24: ('S1', 'PTS/60'),
 25: ('S1', 'S/60'),
 26: ('S1', 'oiGF/60'),
 27: ('S1', 'C/60'),
 28: ('S1', 'F/60'),
 29: ('S1', 'PS'),
 30: ('S2', 'Name'),
 31: ('S2', 'Season'),
 32: ('S2', 'Fantasy Points'),
 33: ('S2', 'Position'),
 34: ('S2', 'Age'),
 35: ('S2', 'GP'),
 36: ('S2', 'ATOI'),
 37: ('S2', '+/-'),
 38: ('S2', 'PTS'),
 39: ('S2', 'G'),
 40: ('S2', 'S'),
 41: ('S2', 'TSA'),
 42: ('S2', 'SThr%'),
 43: ('S2', 'PIM'),
 44: ('S2', 'HIT'),
 45: ('S2', 'BLK'),
 46: ('S2', 'TK'),
 47: ('S2', 'GV')

In [23]:
# Check min/max values and whether the column data is a string
dat=pd.DataFrame(nn_XdataYear[2009])
print(f'Target FP (Max,Min) = ({np.max(nn_YdataYear[2009][0])},{np.min(nn_YdataYear[2009][0])}) ')
print(f'Target PPG (Max,Min) = ({np.max(nn_YdataYear[2009][1])},{np.min(nn_YdataYear[2009][1])}) ')
for c in dat.columns:
    if isinstance(dat[c][0],str):
        print(f'column {c} is a str: {sel_col_dict[c]}')
    else:
        print(f'column {c}, (Max, Min) = ({np.max(dat.loc[:,c])},{np.min(dat.loc[:,c])})')

Target FP (Max,Min) = (10.627130010669106,1.01) 
Target PPG (Max,Min) = (10.591260404280618,0.83) 
column 0 is a str: ('S1', 'Name')
column 1, (Max, Min) = (9,9)
column 2, (Max, Min) = (14.774683544303798,0.5121951219512195)
column 3, (Max, Min) = (1.0,0.0)
column 4, (Max, Min) = (40.0,18.0)
column 5, (Max, Min) = (1.0121951219512195,0.15853658536585366)
column 6 is a str: ('S1', 'ATOI')
column 7, (Max, Min) = (0.45121951219512196,-0.4146341463414634)
column 8, (Max, Min) = (1.39,0.0)
column 9, (Max, Min) = (0.76,0.0)
column 10, (Max, Min) = (6.68,0.07)
column 11, (Max, Min) = (12.974683544303797,0.0)
column 12, (Max, Min) = (0.79,0.0)
column 13, (Max, Min) = (3.93,0.0)
column 14, (Max, Min) = (4.564102564102564,0.08571428571428572)
column 15, (Max, Min) = (3.268292682926829,0.03571428571428571)
column 16, (Max, Min) = (1.1774193548387097,0.0)
column 17, (Max, Min) = (1.3970588235294117,0.0)
column 18, (Max, Min) = (0.684,0.27)
column 19, (Max, Min) = (0.18600000000000003,-0.2880000000

In [24]:
def sel01_data_Clean(data, yTarget):
    rows_to_drop=[]
    for r in data.index:
        # Convert ATOI string to integer of seconds, measured in hours (60mins)
        if isinstance(data.loc[r,6], str):
            data.loc[r,6]=sum([a*b for a,b in zip([60,1], map(int, data.loc[r,6].split(':')))])/3600
        else:
            rows_to_drop.append(r)
        if isinstance(data.loc[r,36], str):
            data.loc[r,36]=sum([a*b for a,b in zip([60,1], map(int, data.loc[r,36].split(':')))])/3600
        else:
            rows_to_drop.append(r)
    
    data=data.drop(30, axis=1) # Name duplicate
    data=data.drop(31, axis=1) # S2 season ID
    data=pd.DataFrame(nan_to_zero(np.array(data)))
    data.insert(0, 'Target PPG', pd.DataFrame(yTarget)[1])
    data.insert(0, 'Target FPPG', pd.DataFrame(yTarget)[0])
    data=data.drop(rows_to_drop, axis=0)
    
    return data

In [25]:
# Obtain cleaned data
start_time=time.time()
all_cleaned_data=sel01_data_Clean(pd.DataFrame(nn_XdataYear[2009]),nn_YdataYear[2009])
for i in range(2010,2021):
    print(i)
    temp=sel01_data_Clean(pd.DataFrame(nn_XdataYear[i]),nn_YdataYear[i])
    all_cleaned_data=pd.concat([all_cleaned_data, temp])
all_cleaned_data.index=range(len(all_cleaned_data.index))
print(f'Finished cleaning data in {time.time()-start_time}s')

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Finished cleaning data in 1.2511441707611084s


In [26]:
print(all_cleaned_data.isna().sum().sum())
for i,r in enumerate(np.array(all_cleaned_data)):
    for j,c in enumerate(r):
        if not isinstance(c, str):
            if math.isnan(c):
                print(i,j)
                print(r)

0


In [27]:
all_cleaned_data

,Target FPPG,Target PPG,0,1,2,3,4,5,6,7,...,48,49,50,51,52,53,54,55,56,57
0,10.627130,1.01,Zach Parise,9,11.34878,1.0,24.0,1.0,0.3125,0.365854,...,0.634,0.16,0.1,0.71,2.7,10.9,3.7,28.3,21.7,8.9
1,10.591260,0.83,Patrik Elias,9,9.237662,1.0,32.0,0.939024,0.309444,0.219512,...,0.631,0.146,0.085,0.622,2.4,11.6,3.1,27.0,21.0,6.5
2,6.976437,0.75,Jamie Langenbrunner,9,8.050617,1.0,33.0,0.987805,0.301667,0.304878,...,0.586,0.076,0.09,0.637,2.1,7.8,2.9,18.9,13.2,4.0
3,7.585366,0.82,Travis Zajac,9,7.20122,1.0,23.0,1.0,0.310833,0.402439,...,0.615,0.123,0.075,0.643,1.5,6.8,2.5,25.1,17.9,2.7
4,11.554045,0.75,Brian Gionta,9,7.125926,1.0,30.0,0.987805,0.282778,0.146341,...,0.613,0.129,0.08,0.657,2.1,10.3,2.8,23.2,18.4,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6371,6.768388,0.45,Joe Thornton,20,3.048571,1.0,40.0,0.853659,0.258333,-0.231707,...,0.6,0.083,0.11,0.599,2.7,4.8,4.0,23.5,16.7,4.8
6372,4.524138,0.17,Marcus Sorensen,20,2.275758,1.0,27.0,0.804878,0.230833,-0.146341,...,0.499,-0.054,0.1,0.537,1.7,5.7,3.0,2.1,-0.2,3.1
6373,3.445829,0.12,Marc-Edouard Vlasic,20,3.051429,0.0,32.0,0.853659,0.338333,-0.121951,...,0.456,-0.13,0.103,0.364,1.0,4.0,2.9,-9.5,-7.5,3.2
6374,4.070970,0.24,Dylan Gambrell,20,2.154,1.0,23.0,0.609756,0.199167,-0.097561,...,0.43,-0.049,0.045,0.459,0.0,5.2,0.9,-7.8,-9.6,-0.1


In [28]:
import dill
dill_file=open("all_data_adv_v1","wb")
dill_file.write(dill.dumps(all_cleaned_data))
dill_file.close()

In [29]:
all_cleaned_test=sel01_data_Clean(pd.DataFrame(nn_XdataYear[2021]),nn_YdataYear[2020])
dill_file=open("all_data_adv_test","wb")
dill_file.write(dill.dumps(all_cleaned_test))
dill_file.close()